In [1]:
import pandas as pd


In [2]:
#function to read excel_file 
def read_file(file_name):
    file = pd.ExcelFile(file_name)
    list = []
    
    #reading data in each sheet of the file and storing in a list
    for sheet_name in file.sheet_names:
        df = pd.read_excel(file, sheet_name = sheet_name)
        #averaging the values to the minute granularity 
        df['Date'] = pd.to_datetime(df['Date (UTC)']).dt.floor('min')
        df = df.drop(columns = ['Date (UTC)'])
        average_min = df.groupby('Date').mean()
        average_min = average_min.reset_index()
        average_min['Date'] = pd.to_datetime(average_min['Date'])
        list.append(average_min)
    return list 

#function to merge all the sheets' dataframe into one dataframe
def merge(list):
    #create a dataframe from the list of dataframes
    result = list[0]
    
    #merging all the dataframes based on the common date
    for index, data_frame in enumerate (list):
        if index < len(list) - 1:
            result = result.merge(list[index + 1], on = 'Date', how = 'inner')
    
    #changing columns' name
    new_cols = {}
    for column in result.columns:
        new_column = column.replace(' for PRODSQL\\LOCAL','').replace('SQL Server: ','').replace(' ','_').lower()
        new_cols[column] = new_column
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > M: (SQL Data 1)'] = 'disk_avg_read_time_1'
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > N: (SQL Data 2)'] = 'disk_avg_read_time_2'
    new_cols['Disk avg. read time for prodsql1-vm.wwwoodproducts.com > I: (SQL Index)'] = 'index1'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > M: (SQL Data 1)'] = 'disk_avg_write_time_1'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > N: (SQL Data 2)'] = 'disk_avg_write_time_2'
    new_cols['Disk avg. write time for prodsql1-vm.wwwoodproducts.com > I: (SQL Index)'] = 'index2'
    result.rename(columns = new_cols, inplace = True )
    
    return result

#function to handling data
def data_process(data_frame):
    #converting the Date (UTC) to Date(CST)
    from datetime import datetime
    import pytz
    utc_timezone = pytz.utc
    cst_timezone = pytz.timezone('America/Chicago')
    data_frame['date'] = data_frame['date'].dt.tz_localize('UTC').dt.tz_convert('US/central').dt.tz_localize(None)
    
    #converting Free Memory metric from Bytes to GB
    data_frame['free_memory'] = data_frame['free_memory']/1024**3
    
    #Multiplying the compilations/batch by 100
    data_frame['compilations/batch'] = data_frame['compilations/batch']*100
    
    #Combine Disk avg read time sql data 1 and Disk avg read time sql data 2 into one column Disk avg read time sql data using a SUM
    data_frame['disk_avg_read_time_sql_data']  = data_frame['disk_avg_read_time_1']+data_frame['disk_avg_read_time_2']
    return data_frame


In [3]:
#reading files, concatenating and writing to csv
def reading(file_name):
    list_of_file = read_file(file_name)
    df = merge(list_of_file)
    df = data_process(df)
    return df

datasetA1 = reading('1st_day_A.xlsx')
datasetA2 = reading('2nd_day_A.xlsx')
datasetA3 = reading('3rd_day_A.xlsx')
datasetA4 = reading('4th_day_A.xlsx')
datasetA5 = reading('5th_day_A.xlsx')

datasetA = pd.concat([datasetA1,datasetA2,datasetA3,datasetA4,datasetA5])
datasetA.to_csv('A.csv')

datasetB1 = reading('1st_day_B.xlsx')
datasetB2 = reading('2nd_day_B.xlsx')
datasetB3 = reading('3rd_day_B.xlsx')
datasetB4 = reading('4th_day_B.xlsx')
datasetB5 = reading('5th_day_B.xlsx')

datasetB = pd.concat([datasetB1,datasetB2,datasetB3,datasetB4,datasetB5])
datasetB.to_csv('B.csv')

In [4]:
datasetB3

,date,batch_requests/sec,user_connections,processor_time,free_memory,page_reads/sec,disk_avg_read_time_1,disk_avg_read_time_2,index1,disk_avg_write_time_1,disk_avg_write_time_2,index2,compilations/batch,latch_wait_time,disk_avg_read_time_sql_data
0,2024-04-30 00:01:00,70.859294,894.0,8.917560,13.159554,187.600022,1.81860,0.00000,17.5319,0.00000,0.0000,0.0000,2.628491,1.4240,1.8186
1,2024-04-30 00:02:00,112.109592,900.0,19.481426,12.880081,161.043493,2.65450,2.23330,18.0338,0.00000,0.0000,0.0000,9.150457,3.1520,4.8878
2,2024-04-30 00:03:00,19.638544,910.0,12.058680,13.441589,0.216540,1.99170,2.92580,2.1993,0.62020,0.5863,0.3983,8.821035,0.4584,4.9175
3,2024-04-30 00:04:00,12.645298,907.0,8.445540,13.412392,77.354570,3.16150,3.78430,1.7693,0.45150,0.4528,0.0000,6.455863,0.5395,6.9458
4,2024-04-30 00:05:00,17.395267,912.0,8.752539,13.408791,0.000000,0.70255,0.67725,0.0000,0.18205,0.0000,0.0000,83.524904,0.3890,1.3798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,2024-04-30 23:55:00,15.254624,1108.0,8.540703,12.315128,133.112172,0.00000,0.00000,0.0000,0.26935,0.0000,0.0000,34.485819,1.5069,0.0000
1434,2024-04-30 23:56:00,9.449810,1110.0,8.482197,12.337402,0.000000,0.00000,0.00000,0.0000,0.48470,0.4156,0.0000,7.054674,0.4456,0.0000
1435,2024-04-30 23:57:00,16.891016,1104.0,8.592492,12.350006,0.349814,1.37090,0.00000,0.0000,0.49220,0.4124,0.4858,10.256410,0.3522,1.3709
1436,2024-04-30 23:58:00,15.142681,1098.0,8.507597,12.377419,89.657105,0.00000,0.00000,0.0000,0.41990,0.7419,0.0000,7.038123,0.8646,0.0000


In [5]:
#Reading data from csv file and filtering results
dataA = pd.read_csv('A.csv')
dataB = pd.read_csv('B.csv')

dataA['date'] = pd.to_datetime(dataA['date'])
dataA_filtered = dataA[(dataA['date'].dt.hour>=5) & (dataA['date'].dt.hour<= 16)]
dataA_filtered.to_csv("[Daytime]A.csv", index=False)

dataB['date'] = pd.to_datetime(dataB['date'])
dataB_filtered = dataB[(dataB['date'].dt.hour>=5) & (dataB['date'].dt.hour<= 16)]
dataB_filtered.to_csv("[Daytime]B.csv", index=False)

